In [28]:
import os
try:
    from google.colab import drive
    COLAB = True
    print("Estoy en Google CoLab")
    %tensorflow_version 2.x
    !pip install livelossplot
except:
    os.environ["CUDA_VISIBLE_DEVICES"]="0" 
    print("No estoy en Google CoLab")
    COLAB = False

Estoy en Google CoLab


# Autoencoders

In [29]:
# Cargamos las librerías necesarias para la práctica
import keras
from keras import models
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, Flatten, LeakyReLU, Activation, Dropout
from keras.layers import Layer, Reshape, Conv2DTranspose, Lambda, ReLU, BatchNormalization
from keras.metrics import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
try:
  from livelossplot import PlotLossesKerasTF
except:
  !pip install livelossplot
  from livelossplot import PlotLossesKerasTF
from keras.utils.vis_utils import plot_model
from keras.datasets import mnist, cifar10
from tqdm import tqdm
from io import BytesIO, StringIO
import inspect
import numpy as np
import pathlib
import cv2
import matplotlib
import os
import random
import re
import tensorflow as tf
# %matplotlib inline
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = (15,5) # Para el tamaño de la imagen
matplotlib.rcParams['figure.figsize'] = (15,5)

import seaborn as sns
import pandas as pd
# import utils

## Introducción

Hasta el momento se han visto aplicaciones de aprendizaje supervisado con redes neuronales (convolucionales y recurrentes). Esto es, para cada muestra en nuestro conjunto de datos teníamos asociada una etiqueta o respuesta esperada.

En este tema vamos a trabajar en aplicaciones de aprendizaje no supervisado. Ahora no tenemos la etiqueta asociada a cada muestra, lo cuál suele ser muy costoso de conseguir. La objetivo principal del aprendizaje no supervisado es evitar el uso de muestra equitadas o al menos ayudar a reducir el tamaño de las muestras etiquetas.

La estructura en deep learning por excelencia para aprendizaje no supervisado es el **autoencoder**. Esta red, fue diseñada para buscar nuevas representaciones de las entradas. Para ello se divide la red en dos partes, el codificador y el decodificador. El **codificador** llevará a cabo una compresión de la entrada hasta generar un vector (llamado **vector latente**), representación de nuestra entrada. El **decodificador**, tendrá que usar este vector latente para *reconstruir* la imagen original de entrada.

<img src='https://hdvirtual.us.es/discovirt/index.php/s/YDRYPdrLzAwCS6Q/preview' width=70% />

El vector latente de es menor dimensionalidad que la entrada. Es decir, si estamos trabajando con imágenes de 200x200 el vector debe ser de una dimensión inferior a 40000 elementos. Esta reducción de dimensionalidad obliga al codificador a quedarse con la información más importante y representativa que permita al decodificador a llevar una reconstrucción lo más fiel posible.

En cierto modo, tenemos dos redes independientes (el codificador y el decodificador) que trabajan para alcanzar un bien común. El el siguiente tema, sobre redes generativas adversarias, veremos que tambien se sigue este enfoque de varias redes colaborando para llevar a cabo una tarea objetivo. 


## Nuestro primer autoencoder en keras

Vamos a crear nuestro primer autoencoder en keras. Para ello, usaremos el conjunto de datos MNIST. Vamos a entrenar nuestro autoencoder para que aprenda una representación de cada dígito en un espacio bidimensional y después visualizaremos las representaciones aprendidas en un gráfico 2D.

Puesto que los autoencoders tiene una estructura bien definida, nos podemos crear una clase para que nos facilite su creación:

```
AE = Autoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64,64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_filters = [64,64,32,1]
    , decoder_conv_kernel_size = [3,3,3,3]
    , decoder_conv_strides = [1,2,2,1]
    , z_dim = 2)
```


Vamos a empezar agregando el código de creación del codificador:


In [30]:
class Autoencoder(tf.keras.Model):
    
    def __init__(self, input_dim, encoder_conv_filters, encoder_conv_kernel_size, encoder_conv_strides,
                 decoder_conv_filters, decoder_conv_kernel_size, decoder_conv_strides, z_dim):
        super(Autoencoder, self).__init__()
        
        self.input_dim = input_dim
        self.encoder_conv_filters = encoder_conv_filters
        self.encoder_conv_kernel_size = encoder_conv_kernel_size
        self.encoder_conv_strides = encoder_conv_strides
        self.decoder_conv_filters = decoder_conv_filters
        self.decoder_conv_kernel_size = decoder_conv_kernel_size
        self.decoder_conv_strides = decoder_conv_strides
        self.z_dim = z_dim
        
        self.encoder = self.__create_encoder()
        
    def __create_encoder(self):

        encoder_input = Input(shape=self.input_dim, name='encoder_input') #(1)
        x = encoder_input
        for i in range(len(self.encoder_conv_filters)):
            conv_layer = Conv2D(
                filters = self.encoder_conv_filters[i],
                kernel_size = self.encoder_conv_kernel_size[i],
                strides = self.encoder_conv_strides[i],
                padding = 'same',
                name = 'encoder_conv_' + str(i)
            )
 
            x = conv_layer(x) #(2)
            x = LeakyReLU()(x)

        shape_before_flattening = K.int_shape(x)[1:]
        x = Flatten()(x) #(3)
        
        encoder_output= Dense(self.z_dim, name='encoder_output')(x) #(4)
        
        return Model(encoder_input, encoder_output) #(5)

   1. Definimos la entrada del autoencoder (en nuestro caso una imagen)
   2. Añadimos la capa convolucionales secuencialmente.
   3. Aplanamos la salida de la última capa convolucional para crear un vector.
   4. Añadimos una capa densa (totalmente conectada) para crear nuestro vector latente de 2 dimensiones.
   5. Creamos el modelo de Keras que define el codificador cuya entrada es la imagen y salida el vector latente.
    
Vamos a probarlo:

In [31]:
AE = Autoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64,64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_filters = [64,64,32,1]
    , decoder_conv_kernel_size = [3,3,3,3]
    , decoder_conv_strides = [1,2,2,1]
    , z_dim = 2)

AE.encoder.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder_conv_0 (Conv2D)     (None, 28, 28, 32)        320       
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 28, 28, 32)        0         
                                                                 
 encoder_conv_1 (Conv2D)     (None, 14, 14, 64)        18496     
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 encoder_conv_2 (Conv2D)     (None, 7, 7, 64)          36928     
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 7, 7, 64)          0   

Podemos cambiar el número de capas convolucionales simplemente añadiendo elementos a las listas. **Se recomienda experimentar con los parámetros para entender mejor como afecta la arquitectura al número de parámetros de cada capa, el rendimiento del modelo y los tiempos de entrenamiento.**

Continuamos ahora con el decodificador. Éste suele ser un *espejo* del codificador excepto por el tipo de capas convolucionales. El decodificador tendrá que mapear el vector latente *z* a las dimensiones de una imagen. Para ello usaremos tambien capas convolucionales para obtener dicha imagen. La salida de este red deberá ser la entrada del codificador.

Para ello debemos aplicar una operación de deconvolución o las traspuesta del aconvolución:

<img src='https://hdvirtual.us.es/discovirt/index.php/s/5C3ZgtpxstrDjwa/preview' />

En Keras tenemos la clase `Conv2DTranspose` que define la funcionalidad de esta capa.

Aunque hemos dicho que el decodificador suele ser una imagen espejo del codificador, no tiene porqué ser así. El decoficador puede tener cualquier estructura pero siempre debe cumplir la condición de que su salida debe tener las misma dimensiones que la entrada del codificador. Esta condición es necesaria para poder definir la función de coste comparando pixel a pixel la entrada del codificador con la salida del decodificador.

Vamos ahora a añadir el método de creación del decodificador a nuestra clase `Autoencoder`:

In [32]:
class Autoencoder(tf.keras.Model):
    
    def __init__(self, input_dim, encoder_conv_filters, encoder_conv_kernel_size, encoder_conv_strides,
                 decoder_conv_filters, decoder_conv_kernel_size, decoder_conv_strides, activation, z_dim,
                 use_batch_normalization=False, dropout = None):
        super(Autoencoder, self).__init__()
        
        self.input_dim = input_dim
        self.encoder_conv_filters = encoder_conv_filters
        self.encoder_conv_kernel_size = encoder_conv_kernel_size
        self.encoder_conv_strides = encoder_conv_strides
        self.decoder_conv_filters = decoder_conv_filters
        self.decoder_conv_kernel_size = decoder_conv_kernel_size
        self.decoder_conv_strides = decoder_conv_strides
        self.z_dim = z_dim
        self.activation = activation
        self.use_batch_normalization = use_batch_normalization
        self.dropout = dropout
        
        self.encoder, encoder_input, encoder_output = self.__create_encoder() 
        self.decoder = self.__create_decoder()
        self.model = Model(encoder_input, self.decoder(encoder_output))
        
    def __create_encoder(self):

        encoder_input = Input(shape=self.input_dim, name='encoder_input') 
        x = encoder_input
        for i in range(len(self.encoder_conv_filters)):
            conv_layer = Conv2D(
                filters = self.encoder_conv_filters[i],
                kernel_size = self.encoder_conv_kernel_size[i],
                strides = self.encoder_conv_strides[i],
                padding = 'same',
                activation = self.activation if isinstance(self.activation, str) else None,
                name = 'encoder_conv_' + str(i)
            )
 
            x = conv_layer(x) #(2)
    
            if inspect.isclass(self.activation) and Layer in self.activation.__bases__:
                x = self.activation()(x)
                
            if self.use_batch_normalization:
                x = BatchNormalization()(x)

            if self.dropout:
                x = Dropout(self.dropout)(x)                

        self.__shape_before_flattening = K.int_shape(x)[1:]
        x = Flatten()(x) #(3)
        
        encoder_output = self._create_latent_vector(encoder_input, x)
        
        return Model(encoder_input, encoder_output), encoder_input, encoder_output #(5)
        
    def _create_latent_vector(self, encoder_input, x):
        encoder_output= Dense(self.z_dim, name='encoder_output')(x) 
        return encoder_output
    
    
    def __create_decoder(self):

        decoder_input = Input(shape=(self.z_dim,), name='decoder_input') #(1)
        
        x = Dense(np.prod(self.__shape_before_flattening))(decoder_input) #(2)
        
        
        x = Reshape(self.__shape_before_flattening)(x) #(3)
        
        for i in range(len(self.decoder_conv_filters)):
            activation = self.activation if isinstance(self.activation, str) else None
            conv_layer = Conv2DTranspose(
                filters = self.decoder_conv_filters[i],
                kernel_size = self.decoder_conv_kernel_size[i],
                strides = self.decoder_conv_strides[i],
                padding = 'same',
                activation = activation if i < len(self.decoder_conv_filters)-1 else 'sigmoid',
                name = 'decoder_conv_' + str(i)
            )
 
            x = conv_layer(x) #(4)
    
            if i < len(self.decoder_conv_filters)-1:
                if inspect.isclass(self.activation) and Layer in self.activation.__bases__:
                    x = self.activation()(x)

                if self.use_batch_normalization:
                    x = BatchNormalization()(x)

                if self.dropout:
                    x = Dropout(self.dropout)(x)                
            
            
        decoder_output = x
        
        return Model(decoder_input, decoder_output) #(6)
    

   1. Definimos la entrada del decodificador (la entrada es el vector latente)
   2. Conectamos la entrada a una capa densa para expandir el vector.
   3. Debemos convertir el vector a una estructura 2D para que sirva de entrada a la primera capa convolucional.
   4. Vamos estacando la capas convolucionales.
   5. Debemos controla si es la última capa convolucional. En ese caso la activación debe ser una sigmoide.
   6. Creamos el modelo Keras que define el decodificador que toma como entrada el vector latente y como salida una imagen de las mismas dimensiones que la imagen de entrada.
    
Vamos a probarlo:

In [33]:
AE = Autoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64,64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_filters = [64,64,32,1]
    , decoder_conv_kernel_size = [3,3,3,3]
    , decoder_conv_strides = [1,2,2,1]
    , activation = 'relu'
    , z_dim = 2)

AE.decoder.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense_2 (Dense)             (None, 3136)              9408      
                                                                 
 reshape_2 (Reshape)         (None, 7, 7, 64)          0         
                                                                 
 decoder_conv_0 (Conv2DTrans  (None, 7, 7, 64)         36928     
 pose)                                                           
                                                                 
 decoder_conv_1 (Conv2DTrans  (None, 14, 14, 64)       36928     
 pose)                                                           
                                                                 
 decoder_conv_2 (Conv2DTrans  (None, 28, 28, 32)       184

Para poder entrenar el autoencoder debemos unir el codificador y del decodificador. Para ello hemos modificador el método `__init__` en sus última líneas:

```
        self.encoder, encoder_input, encoder_output = self.__create_encoder() #(1)
        self.decoder = self.__create_decoder()                                
        self.model = Model(encoder_input, self.decoder(encoder_output))       #(2) 
```

   1. La entrada del autoencoder será la misma que la entrada del codificador. Además la salida del codificador será la entrada decodificador. Hemos modificador el método `__create_encoder` para que nos devuelva esos tensores.
   2. Creamos el modelo Keras del autoencoder cuya entrada, como ya hemos dicho, es la entrada del codificador y su salida es la salida que dará el decodificador al darle como entrada al salida del codificador.

In [34]:
AE.model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder_conv_0 (Conv2D)     (None, 28, 28, 32)        320       
                                                                 
 encoder_conv_1 (Conv2D)     (None, 14, 14, 64)        18496     
                                                                 
 encoder_conv_2 (Conv2D)     (None, 7, 7, 64)          36928     
                                                                 
 encoder_conv_3 (Conv2D)     (None, 7, 7, 64)          36928     
                                                                 
 flatten_5 (Flatten)         (None, 3136)              0         
                                                                 
 encoder_output (Dense)      (None, 2)                 627

Ahora que ya tenemos nuestro modelo creado, podemos compilarlo usando una optimizador y una función de coste. Respecto a la función de coste tenemos varias opciones. Podemos usar RMSE, *binary cross entropy* o SSIM (similaridad estructural). La función de coste *binary cross entropy* realiza una comparación pixel a pixel entre la imagen original y la reconstruida penalizando mucho las malas predicciones, lo que hace que la red tienda a predecir el punto medio del rango. Esto da lugar a imágenes muy claras (con mucho gris). 

En nuestro caso usaremos RMSE como función de coste y SSIM como métrica. Es recomendable **probar binary cross entropy y SSIM como funciones de coste para ver su efecto**. 

In [36]:
ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))
AE.model.compile(optimizer=Adam(), loss = 'mean_squared_error', metrics=[ssim_loss])

NameError: ignored

El último paso es entrenar nuestra autoencoder. Para ello necesitamos el conjunto de datos MNIST que viene con keras.

In [ ]:
# MNIST dataset
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()

image_size = x_train_mnist.shape[1]
x_train_mnist = np.reshape(x_train_mnist, [-1, image_size, image_size, 1])
x_test_mnist = np.reshape(x_test_mnist, [-1, image_size, image_size, 1])
x_train_mnist = x_train_mnist.astype('float32') / 255
x_test_mnist = x_test_mnist.astype('float32') / 255



In [ ]:
AE.model.fit(x=x_train_mnist, y=x_train_mnist, 
             validation_data=(x_test_mnist, x_test_mnist),
             batch_size=128, shuffle=True, epochs=50,
             callbacks=[PlotLossesKerasTF()])

In [ ]:
AE.model.save_weights('AE_mnist_weights.h5')

## Visualizando la reconstrucción

Ahora que tenemos el autoencoder entrenado podemos comparar las imágenes de entrada y su reconstrucción:

In [ ]:
n = 25
imgs = [x_test_mnist[i:i+5] for i in range(0, 25, 5)]
draw2compare(*imgs, figsize=(10, 10))

In [ ]:
n = 25
decoded_imgs = AE.model.predict(x_test_mnist[:n])
imgs = [decoded_imgs[i:i+5] for i in range(0, 25, 5)]
draw2compare(*imgs, figsize=(10, 10))

Aquí vemos algunas cosas interesantes:
   1. Algunos dígitis tienen reconstrucción. No es algo preocupante. Estamos usando una vector latente de 2D para llevar a cabo la visualización del espacio latente. **Se recomienda entrenar al autoencoder con un vector de mayor dimensión y ver la mejora en la reconstrucción**.
   2. Algunos dígitos se confunden con otros.
   3. En general la reconstrucciones están difuminadas.
   4. Podemos decir que el vector latente ha capturado ciertas características globales para poder reconstruir el dígito, pero no contiene información sobre detalles.
   
   


### Visualizando el espacio latente

Para visualizar el espacio latente solo tenemos que pasar las muestras por el codificador y representarlas con un scatter plot. Como tenemos identificado que dígito es cada muestra, lo usaremos para identificar cada punto con un color:

In [ ]:
# función en utils.py
show_latent_space(AE, x_test_mnist)

## Muestreando el espacio latente.

Hemos generado un espacio dimensional inferior de representaciones (que naturalmente es un espacio vectorial) donde cada punto puede ser mapeado a imagen realista. Ahora podemos usar nuestro generador para crear nuevas muestras. Solo tenemos que pasar al decodificador nuevos vectores:

In [ ]:
len(decoded_imgs[0])

In [ ]:
# creamos una lista de vectores (puntos en nuevo espacio latente)
#vectors = np.array([[-7, 5], [-5, 10], [0,0], [10, 0], [10, -5], [3, 5], [1, -5]])
vectors = (np.random.rand(25, 2) * 40) - 20

# usamos el decodificador para obtener la imágenes reconstruidas a partir de los vectores
decoded_imgs = AE.decoder.predict(vectors)
print(decoded_imgs.shape)
decoded_imgs = [decoded_imgs[i:i+5] for i in range(0, 25, 5)]
draw2compare(*decoded_imgs)

# mostramos el espacio latente resaltando los puntos que hemos muestreado
show_latent_space(AE, x_test_mnist, vectors)


De manera similar podemos crear una visualización para tener una mejor idea de como se distribuye nuestro espacio latente. Solo tenemos que muestrear a lo largo del espacio y crear un grid de imágenes:

In [ ]:
sample_latent_space(AE, (15, -10), (-10, 10), n=30)

Como vemos hay gran cantidad de imagenes que no parecen dígítos. Esto es debido a que hemos hecho una barrido lineal, lo que nos habrá llevado a muchas zonas vacías del espacio latente que no tienen ninguna representación cercana en el conjunto de entrenamiento. 

El segundo problema (digitos con una pésima reconstrucción) lo  podemos mitigar aumentand la dimensión del espacio latente a costa de perder poder de representación:

In [ ]:
AE5 = Autoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64,64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_filters = [64,64,32,1]
    , decoder_conv_kernel_size = [3,3,3,3]
    , decoder_conv_strides = [1,2,2,1]
    , activation = 'relu'
    , z_dim = 5)

AE5.model.compile(optimizer=Adam(), loss = 'mean_squared_error', metrics=[ssim_loss])

AE5.model.fit(x=x_train_mnist, y=x_train_mnist, 
             validation_data=(x_test_mnist, x_test_mnist),
             batch_size=128, shuffle=True, epochs=50,
             callbacks=[PlotLossesKerasTF()])

In [ ]:
n = 25
decoded_imgs = AE5.model.predict(x_test_mnist[:n])
imgs = [decoded_imgs[i:i+5] for i in range(0, 25, 5)]
draw2compare(*imgs, figsize=(10, 10))

# Algunas aplicaciones

### Denoising autoencoders 

Los *denoising autoencoders* o autoencoders para eliminación/reducción de ruido son una aplicación particular de los autoencoders. Desde el punto de vista de la estructura de la red no tienen nada de particular. Lo única particularidad es el modo de entrenamiento, en concreto, los datos de entrada. Así que nos vamos a crear nuestra red como en el ejemplo anterior:

In [ ]:
DAE_MNIST = Autoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64,64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_filters = [64,64,32,1]
    , decoder_conv_kernel_size = [3,3,3,3]
    , decoder_conv_strides = [1,2,2,1]
    , activation = 'relu'
    , z_dim = 16)

DAE_MNIST.model.compile(optimizer=Adam(), loss = 'mean_squared_error', metrics=[ssim_loss])

Vamos a seguir jugando con datos de los que no tenemos etiquetas asociadas a las muestras (o al menos no las vamos a usar en la fase de entrenamiento). Lo que cambia en esta aplicación es que vamos a aplicar una distorción a la imagen de entrada, pero la imagen que esperamos que reconstruya será la imagen limpia:

<img src='https://hdvirtual.us.es/discovirt/index.php/s/jYsMLB5WTRS9Rg7/preview' width='70%'>

Vamos a seguir con el conjunto de datos MNIST. En primer lugar vamos a definir una función para agregar ruido a un conjunto de datos:

In [ ]:
def add_noise(data, center=0.5, std=0.5, hard=False):
    # ruido usando una distribución normal centrada en 0.5 y con std 0.5
    noise = np.random.normal(loc=0.5, scale=std, size=data.shape)
    
    if hard:
        return np.clip(data + noise, 0., 1.)
    else:
        img2 = data*2
        n4 = np.clip(np.where(img2 <= 1, (img2*(1 + noise*0.4)), (1-img2+1)*(1 + noise*0.4)*-1 + 2)/2, 0,1)
        return n4

In [ ]:
draw2compare(add_noise(x_test_mnist[:10], hard=True), x_test_mnist[:10])

Es una función muy sencilla que agrega un ruido aleatorio usando una distribución normal centrada en 0.5 y con desviación estanda 0.5. Se puede apreciar que las imágenes son bastante ruidosas. 

Por tanto, a la hora de entrenar le daremos a la red las imágenes de le primera fila y para computar el error cometido le daremos las imágenes de segunda fila.

In [ ]:
x_train_noisy = add_noise(x_train_mnist, hard=True)
x_test_noisy = add_noise(x_test_mnist, hard=True)

DAE_MNIST.model.fit(x=x_train_noisy, y=x_train_mnist, 
             validation_data=(x_test_noisy, x_test_mnist),
             batch_size=128, shuffle=True, epochs=50,
             callbacks=[PlotLossesKerasTF()])

In [ ]:
DAE_MNIST.model.save_weights('dae_mnist_weights.h5')

Una vez finalizado el entrenamiento podemos ver la calidad de nuestro reductor de ruido:

In [ ]:
noise_samples = add_noise(x_test_mnist[:10], hard=True)
denoise_samples = DAE_MNIST.model.predict(noise_samples)

draw2compare(x_test_mnist[:10], noise_samples, denoise_samples)

Parece muy efectivo con este conjunto de datos. ¿Y si probamos con un conjunto de datos "más real"?

### Denoising autoencoder con CIFAR10

El conjunto de datoa CIFAR10 esta compuesto por imágenes RGB de tamaño 32x32. Siguen siendo pequeñas, pero al menos vamos a probar con imágenes más complejas y a color.

Al igual que con MNIST solo tenemos que leer el conjunto de datos con Keras y escalar las imágenes a valores entre 0 y 1:

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

image_size = x_train.shape[1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

En este caso vamos a probar con un ruido un poco más sutíl. Aplicaremos un ruido gausiano centrado en 0 y con desviación 0.4:

In [ ]:
draw2compare(x_train[:10], add_noise(x_train[:10], center=0, std=0.4))

Algunos cambios con respecto al ejemplo con MNIST:
   1. Ahora el decoficador debe generar una imagen con 3 canales (RGB) por lo que la última capa convolucional debe generar 3 mapas (filters = 3). Anteriormente, como eran imágenes en escala de grises, este valor era 1.
   2. El tamaño del vector latente ahora es mucho más grande. Queremos eliminar el ruido, pero no queremos perder excesiva información de la imagen de entrada.
   3. `ReduceLROnPlateau` implementa una reducción progresiva del learning rate cuando no se produce mejora.

In [ ]:
DAE_CIFAR10 = Autoencoder(
        input_dim = (32,32,3)
        , encoder_conv_filters = [32,64,64,64]
        , encoder_conv_kernel_size = [3,3,3,3]
        , encoder_conv_strides = [1,2,2,1]
        , decoder_conv_filters = [64,64,32,3]
        , decoder_conv_kernel_size = [3,3,3,3]         #(1)
        , decoder_conv_strides = [1,2,2,1]
        , activation = 'relu'
        , z_dim = 1024)                                 #(2)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, verbose=1, min_lr=0.5e-6)   #(3)

DAE_CIFAR10.model.compile(optimizer='adam', loss = 'mean_squared_error', metrics=[ssim_loss])

x_train_noisy = add_noise(x_train, center=0, std=0.4)
x_test_noisy = add_noise(x_test,center=0, std=0.4)

DAE_CIFAR10.model.fit(x=x_train_noisy, y=x_train, 
             validation_data=(x_test_noisy, x_test),
             batch_size=128, shuffle=True, epochs=50,
             callbacks=[PlotLossesKerasTF()])

In [ ]:
DAE_CIFAR10.model.save_weights('dae_cifar10_weights.h5')

In [ ]:
noise_samples = add_noise(x_test[:10], center=0, std=0.4)
denoise_samples = DAE_CIFAR10.model.predict(noise_samples)

draw2compare(x_test[:10], noise_samples, denoise_samples)

## Autoencoder para colorear

En esta aplicación vamos a enseñar al autoencoder a colorear imágenes en blanco y negros. El proceso es muy similar al caso anterior. Tomaremos las imágenes de nuestro conjunto de datos y a la entrada le vamos a aplicar una transformación, en este caso la convertiremos a blanco y negro. La salida esperada, que le daremos a la red, será la imagen original con en el caso anterior:

<img src='https://hdvirtual.us.es/discovirt/index.php/s/pkkKpCm25XbjCKH/preview' width='70%' />

Vamos a usar una función que pasará nuestras imágenes en color de CIFAR10 a blanco y negro:

In [ ]:
#from utils import rgb2gray


def rgb2gray(data):
    # grayscale = 0.299*red + 0.587*green + 0.114*blue
    factors = [0.299, 0.587, 0.114]
    return np.dot(data[...,:3], factors).reshape(list(data.shape[:-1]) + [1])
        
    
draw2compare(rgb2gray(x_train[:10]), x_train[:10])

La primera fila serán las imágenes de entrada a nuestro autoencoder y la segunda fila la reconstrucción esperada.

El proceso de entrenamiento es el mismo que en el caso anterior. Únicamente se ha cambiado el punto (3) donde se transforman los datos que vamos a usar de entrada. Sobre la construcción del autoencoder en (1) hemos cambiado la dimensión de entrada. Ahora solo tenemos un canal, ya que la entrada es en escala de grises. En (2) seguimos teniendo 3 canales de salida ya que esperamos la imagen en RGB.

In [ ]:
CAE_CIFAR10 = Autoencoder(
        input_dim = (32,32,1)                          #(1)
        , encoder_conv_filters = [32,64,64,64]
        , encoder_conv_kernel_size = [3,3,3,3]
        , encoder_conv_strides = [1,2,2,1]
        , decoder_conv_filters = [64,64,32,3]
        , decoder_conv_kernel_size = [3,3,3,3]         #(2)
        , decoder_conv_strides = [1,2,2,1]
        , activation = 'relu'
        , z_dim = 1024)                                

CAE_CIFAR10.model.compile(optimizer='adam', loss = 'mean_squared_error', metrics=[ssim_loss])

x_train_gray = rgb2gray(x_train)                     #(3)
x_test_gray = rgb2gray(x_test)

CAE_CIFAR10.model.fit(x=x_train_gray, y=x_train, 
             validation_data=(x_test_gray, x_test),
             batch_size=128, shuffle=True, epochs=20,
             callbacks=[PlotLossesKerasTF()])

# Referencias

 - Generative Deep Learning. Teaching Machines to Paint, Write, Compose and Play. David Foster. O'Reilly
 - Advanced Deep Learning with Keras. Rowel Atienza. Packt.